In [ ]:
!git clone https://github.com/purvi1239/FakeDetectionModel.git

Cloning into 'FakeDetectionModel'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (1481/1481), done.
remote: Compressing objects: 100% (1477/1477), done.
remote: Total 1481 (delta 1), reused 1480 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1481/1481), 22.83 MiB | 22.18 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
%cd /content/FakeDetectionModel


/content/FakeDetectionModel


In [3]:
!pip install tensorflow keras pillow numpy scikit-learn matplotlib


In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


In [5]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
print("Model loaded!")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Model loaded!


In [6]:
def extract_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x, verbose=0)
        return features.flatten()
    except Exception as e:
        print("Error processing:", img_path, str(e))
        return None


In [7]:
logo_folder = "/content/FakeDetectionModel/dataset/Logos"
uploaded_folder = "/content/FakeDetectionModel/dataset/Uploaded"


In [8]:
os.listdir(logo_folder), os.listdir(uploaded_folder)


(['brc-vector-logo-400x400.png',
  'facebook-like-logo-vector-400x400.png',
  'bb-news-logo-vector-free-download.png',
  'Ekstraklasa-logo-400x400.png',
  'Fosters-logo-vector-download-400x400.jpg',
  'state-seal-of-texas-vector-logo-400x400.png',
  'asus-black-vector-logo-400x400.png',
  'art-corel-draw-12-vector-logo-400x400.png',
  'dora-logo-vector-400x400.png',
  'audi-logo-vector-download-400x400.jpg',
  'sgt-peppers-lonely-hearts-club-band-vector-logo-400x400.png',
  'Copyright-symbol-vector-free-download.png',
  'rohs-tested-logo-vector-400x400.png',
  'che-guevara-ernesto-.eps-logo-vector-400x400',
  'suzuki-logo-1.png',
  'honda-bike-vector-400x400.jpg',
  'f-n-logo.png',
  'snapchat-logo-preview-400x400.png',
  'skechers-logo-preview-400x400.png',
  'philips-logo.png',
  'cdc-vector-logo-400x400.png',
  'eibach-springs-.eps-logo-vector-400x400',
  'maserati-logo-vector-download-400x400.jpg',
  'nintendo-ds-vector-logo-400x400.png',
  'playboy-magazine-vector-logo-400x400.png

In [9]:
logo_vectors = {}

for logo_name in os.listdir(logo_folder):
    path = os.path.join(logo_folder, logo_name)
    vec = extract_features(path)
    if vec is not None:
        logo_vectors[logo_name] = vec

print("Extracted vectors for", len(logo_vectors), "logos.")



Extracted vectors for 1471 logos.


In [10]:
for img_name in os.listdir(uploaded_folder):
    img_path = os.path.join(uploaded_folder, img_name)
    uploaded_vec = extract_features(img_path)
    if uploaded_vec is None:
        continue

    print("\nUploaded Image:", img_name)

    best_match = None
    best_score = 0

    for logo_name, logo_vec in logo_vectors.items():
        sim = cosine_similarity([uploaded_vec], [logo_vec])[0][0]

        print("  →", logo_name, ":", round(sim * 100, 2), "% match")

        if sim > best_score:
            best_score = sim
            best_match = logo_name

    print("\nBest Match:", best_match, "| Similarity =", round(best_score*100, 2), "%")

    if best_score > 0.85:
        print("Result → GENUINE ✔️")
    elif best_score > 0.60:
        print("Result → SUSPICIOUS ⚠️")
    else:
        print("Result → FAKE ❌")


In [11]:
test_img = "/content/FakeDetectionModel/dataset/Uploaded/Screenshot 2026-01-24 152323.png"
test_vec = extract_features(test_img)

best_match = None
best_score = 0

for logo_name, logo_vec in logo_vectors.items():
    sim = cosine_similarity([test_vec], [logo_vec])[0][0]
    if sim > best_score:
        best_score = sim
        best_match = logo_name

print("Best Match:", best_match)
print("Similarity:", round(best_score*100, 2), "%")


Best Match: coca-cola-three-bottle-logo-vector-400x400.png
Similarity: 51.1 %


In [12]:
test_img = "/content/FakeDetectionModel/dataset/Uploaded/Screenshot 2026-01-24 153730.png"
test_vec = extract_features(test_img)

best_match = None
best_score = 0

for logo_name, logo_vec in logo_vectors.items():
    sim = cosine_similarity([test_vec], [logo_vec])[0][0]
    if sim > best_score:
        best_score = sim
        best_match = logo_name

print("Best Match:", best_match)
print("Similarity:", round(best_score*100, 2), "%")


Best Match: trek-vector-logo-400x400.png
Similarity: 49.78 %


In [13]:
import pandas as pd
import os

csv_path = "/content/FakeDetectionModel/dataset/LogoDatabase.csv"
df = pd.read_csv(csv_path)

df.head()


,logoName,fileName
0,YouTube,youtube-logo.png
1,YouTube icon vector,YouTube-icon-400x400.png
2,YouTube,youtube-logo-preview-1-400x400.png
3,FC Barcelona,new-fc-barcelona-logo.png
4,YouTube,youtube-eps-version-vector-logo-400x400.png


In [14]:
images_folder = "/content/FakeDetectionModel/dataset/Logos"


In [15]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False)
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = feature_model.predict(img)
    return features.flatten()


/tmp/ipython-input-3324068985.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [16]:
logo_features = []
logo_labels = []
logo_files = []

for _, row in df.iterrows():
    img_path = os.path.join(images_folder, row["fileName"])

    if os.path.exists(img_path):
        vec = extract_features(img_path)
        logo_features.append(vec)
        logo_labels.append(row["logoName"].lower())
        logo_files.append(row["fileName"])
    else:
        print("Missing:", img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━

In [17]:
from google.colab import files
uploaded = files.upload()

test_img_path = list(uploaded.keys())[0]


Saving Screenshot 2026-01-24 153730.png to Screenshot 2026-01-24 153730.png


In [18]:
user_brand = input("Enter the brand name (example: adidas, nike): ").lower()


Enter the brand name (example: adidas, nike): adidas


In [19]:
test_vec = extract_features(test_img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [20]:
filtered_indices = [i for i, b in enumerate(logo_labels) if user_brand in b]

if len(filtered_indices) == 0:
    print("Brand not found in LogoDatabase.csv")
    raise SystemExit


In [21]:
from sklearn.metrics.pairwise import cosine_similarity

best_match = None
best_score = -1

for idx in filtered_indices:
    sim = cosine_similarity([test_vec], [logo_features[idx]])[0][0]

    if sim > best_score:
        best_score = sim
        best_match = logo_files[idx]


In [22]:
print("Best match:", best_match)
print("Similarity:", round(best_score*100, 2), "%")

if best_score > 0.70:
    print("✔ REAL LOGO")
else:
    print("❌ FAKE LOGO")


Best match: Adidas-Trefoil-logo-vector-for-free-download.png
Similarity: 18.88 %
❌ FAKE LOGO


In [23]:
from google.colab import files
uploaded = files.upload()

test_img_path = list(uploaded.keys())[0]


Saving Screenshot 2026-01-24 155915.png to Screenshot 2026-01-24 155915.png


In [24]:
user_brand = input("Enter the brand name (example: adidas, nike): ").lower()


Enter the brand name (example: adidas, nike): adidas


In [25]:
filtered_indices = [i for i, b in enumerate(logo_labels) if user_brand in b]

if len(filtered_indices) == 0:
    print("Brand not found in LogoDatabase.csv")
    raise SystemExit


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

best_match = None
best_score = -1

for idx in filtered_indices:
    sim = cosine_similarity([test_vec], [logo_features[idx]])[0][0]

    if sim > best_score:
        best_score = sim
        best_match = logo_files[idx]


In [27]:
print("Best match:", best_match)
print("Similarity:", round(best_score*100, 2), "%")

if best_score > 0.70:
    print("✔ REAL LOGO")
else:
    print("❌ FAKE LOGO")


Best match: Adidas-Trefoil-logo-vector-for-free-download.png
Similarity: 18.88 %
❌ FAKE LOGO


In [28]:
from google.colab import files
uploaded = files.upload()

test_img_path = list(uploaded.keys())[0]


Saving Screenshot 2026-01-24 155915.png to Screenshot 2026-01-24 155915 (1).png


In [29]:
user_brand = input("Enter the brand name (example: adidas, nike): ").lower()



Enter the brand name (example: adidas, nike): adidas


In [30]:
test_vec = extract_features(test_img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step


In [31]:
filtered_indices = [i for i, b in enumerate(logo_labels) if user_brand in b]

if len(filtered_indices) == 0:
    print("Brand not found in LogoDatabase.csv")
    raise SystemExit


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

best_match = None
best_score = -1

for idx in filtered_indices:
    sim = cosine_similarity([test_vec], [logo_features[idx]])[0][0]

    if sim > best_score:
        best_score = sim
        best_match = logo_files[idx]


In [33]:
print("Best match:", best_match)
print("Similarity:", round(best_score*100, 2), "%")

if best_score > 0.70:
    print("✔ REAL LOGO")
else:
    print("❌ FAKE LOGO")


Best match: Adidas-Trefoil-logo-vector-for-free-download.png
Similarity: 27.02 %
❌ FAKE LOGO


In [34]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_3be1bx7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_3be1bx7
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=c1555242dc890ef3b153bfb8913afc7ddefd8a5932f5b026dd4b34c0f054e78b
  Stored in directory: /tmp/pip-ephem-wheel-cache-eyk6beej/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [35]:
import clip
import torch
from PIL import Image
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 146MiB/s]


In [39]:
import os
import shutil
import pandas as pd

base_path = "/content/FakeDetectionModel/dataset"
logos_path = os.path.join(base_path, "Logos")
target_base = os.path.join(base_path, "original")

csv_path = os.path.join(base_path, "LogoDatabase.csv")
df = pd.read_csv(csv_path)

# Create /original folder if missing
os.makedirs(target_base, exist_ok=True)

moved = 0
missing = 0

for _, row in df.iterrows():
    file_name = row["fileName"]
    brand_name = str(row["logoName"]).lower().strip()

    src = os.path.join(logos_path, file_name)
    brand_folder = os.path.join(target_base, brand_name)
    dst = os.path.join(brand_folder, file_name)

    # Create brand folder if missing
    os.makedirs(brand_folder, exist_ok=True)

    if os.path.exists(src):
        shutil.copy(src, dst)  # use copy to keep original safe (or use move)
        moved += 1
    else:
        print("❌ File not found:", src)
        missing += 1

print("✅ Dataset organized successfully!")
print("Moved files:", moved)
print("Missing files:", missing)


❌ File not found: /content/FakeDetectionModel/dataset/Logos/calvin-klein-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/diga-nro-аs-drogas.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/caterpillar-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/new-instagram-logo-seeklogo.net_-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/converse-all-star-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/brembo-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/converse-shoes-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/chrysler-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/dodge-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/benetton-.eps-logo-vector-400x400.png
❌ File not 

In [37]:
/content/FakeDetectionModel/dataset/original/adidas/


NameError: name 'content' is not defined

In [40]:
import os
import shutil
import pandas as pd

base_path = "/content/FakeDetectionModel/dataset"
logos_path = os.path.join(base_path, "Logos")
target_base = os.path.join(base_path, "original")

csv_path = os.path.join(base_path, "LogoDatabase.csv")
df = pd.read_csv(csv_path)

# Create /original folder if missing
os.makedirs(target_base, exist_ok=True)

moved = 0
missing = 0

for _, row in df.iterrows():
    file_name = row["fileName"]
    brand_name = str(row["logoName"]).lower().strip()

    src = os.path.join(logos_path, file_name)
    brand_folder = os.path.join(target_base, brand_name)
    dst = os.path.join(brand_folder, file_name)

    # Create brand folder if missing
    os.makedirs(brand_folder, exist_ok=True)

    if os.path.exists(src):
        shutil.copy(src, dst)  # use copy to keep original safe (or use move)
        moved += 1
    else:
        print("❌ File not found:", src)
        missing += 1

print("✅ Dataset organized successfully!")
print("Moved files:", moved)
print("Missing files:", missing)


❌ File not found: /content/FakeDetectionModel/dataset/Logos/calvin-klein-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/diga-nro-аs-drogas.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/caterpillar-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/new-instagram-logo-seeklogo.net_-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/converse-all-star-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/brembo-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/converse-shoes-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/chrysler-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/dodge-.eps-logo-vector-400x400.png
❌ File not found: /content/FakeDetectionModel/dataset/Logos/benetton-.eps-logo-vector-400x400.png
❌ File not 

In [41]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-biyre4o2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-biyre4o2
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [42]:
import clip
import torch
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device)


In [43]:
def get_clip_features(img_path):
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.encode_image(image)

    return features.cpu().numpy().flatten()


In [44]:
import os
from sklearn.metrics.pairwise import cosine_similarity

def load_brand_embeddings(brand):
    brand_folder = f"/content/FakeDetectionModel/dataset/original/{brand.lower()}"

    if not os.path.exists(brand_folder):
        print("❌ Brand folder not found:", brand_folder)
        return None, None

    vectors = []
    files = []

    for file in os.listdir(brand_folder):
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(brand_folder, file)
            vec = get_clip_features(img_path)
            vectors.append(vec)
            files.append(file)

    return vectors, files


In [49]:
from google.colab import files

uploaded = files.upload()
customer_image = list(uploaded.keys())[0]
print("Uploaded Image:", customer_image)


Saving Screenshot 2026-01-24 at 3.42.54 PM.png to Screenshot 2026-01-24 at 3.42.54 PM.png
Uploaded Image: Screenshot 2026-01-24 at 3.42.54 PM.png


In [50]:
brand = input("Enter brand name (example: adidas, nike, puma): ").lower()


Enter brand name (example: adidas, nike, puma): nike


In [51]:
def compare_with_brand(customer_image, brand):

    vectors, files = load_brand_embeddings(brand)

    if vectors is None:
        return

    # Feature vector of customer's image
    customer_vec = get_clip_features(customer_image)

    best_score = -1
    best_match = None

    for ref_vec, ref_file in zip(vectors, files):
        sim = cosine_similarity([customer_vec], [ref_vec])[0][0]

        if sim > best_score:
            best_score = sim
            best_match = ref_file

    print("\n🔍 Best Match:", best_match)
    print("Similarity:", round(best_score * 100, 2), "%")

    # Real/Fake threshold optimized for CLIP logos
    if best_score >= 0.40:
        print("✔ REAL LOGO")
    else:
        print("❌ FAKE LOGO")


In [52]:
compare_with_brand(customer_image, brand)



🔍 Best Match: nike-logo-vector-download-400x400.jpg
Similarity: 70.62 %
✔ REAL LOGO
